# Chroma Vector Database
- Chroma는 대규모 언어 모델(LLM) 애플리케이션 구축을 위해 설계된 AI 네이티브 **오픈 소스 벡터 데이터베이스**다.    
- 임베딩 저장소, 쿼리 및 검색 등의 핵심 기능을 제공하여 개발자들이 효율적으로 작업할 수 있도록 돕는다. 
- https://www.trychroma.com/
  
## Chroma의 주요 특징

- **오픈 소스 라이선스** 
  - Apache 2.0 라이선스에 따라 제공되어 누구나 자유롭게 사용하고 수정할 수 있다. 
- **다양한 개발 환경 지원**
  -  Python 및 JavaScript/TypeScript SDK를 지원하여 다양한 Langchain 과 연동하여 활용할 수 있다. 
- **유연한 데이터 저장 옵션**
  -  HTTP 방식, 디스크 저장 방식, 인메모리 방식을 선택하여 데이터를 저장할 수 있어 사용자 입장에서 매우 편리하다. 
- **간편한 사용법** 
  - 설치 및 사용법이 매우 간단하여 빠르게 프로토타입을 개발하고 검증할 수 있다. 

## 설치
- <del>pip로 chromadb 설치시 **windows**에서는 c컴파일러 관련되어 에러가 난다. **conda 를 이용해 설치한다.**</del>
- `conda install conda-forge::chromadb`
- `pip install chromadb`
- `pip install langchain-chroma`

# Chroma API 를 이용해 연동
- https://docs.trychroma.com/

In [1]:
import chromadb

In [2]:
from uuid import uuid4

# 추가할 데이터
document_list = [
        "This is a document about pineapple",
        "This is a document about oranges",
        "This is a document about sports",
        "This is a document about langchain",
]
ids = [str(uuid4()) for _ in range(len(document_list))]
# 디비에 저장할 때 지정할 각 문서들의 ID 생성.

In [ ]:
#  외부 Embedding 모델 
from dotenv import load_dotenv
import chromadb.utils.embedding_functions as embedding_functions
import os

# 환경 변수 로드
print(load_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI 임베딩 함수 객체 생성
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )


True


In [4]:
# collection - Database
# Chroma DB와 연결
import chromadb
client = chromadb.Client()   # InMemory DB (데이터를 메모리에 저장)
# client = chromadb.PersistentClient(path="vector_store/chroma/my_db") # Local 파일에 저장.
# client = chromadb.HttpClient(host="ip주소", port=8000) # 서버로 서비스하는 chromadb에 연결

# Collection(Database)을 생성
collection_name = "test_db"
collection = client.create_collection(
    name=collection_name,
    get_or_create=True, # collection이 있으면 연결, 없으면 생성. 
                        #  (False: 이미 있는 collection이면 Exception)
    metadata={"hnsw:space":"cosine"}, # 코사인 유사도록 계산.
    embedding_function=openai_ef
)


In [9]:
type(collection) # collection 객체구나~

chromadb.api.models.Collection.Collection

In [5]:
######### 데이터 추가
collection.add(documents=document_list, ids=ids)

In [6]:
##### 유사도 검색
result = collection.query(
    query_texts=["deeplearning", "hawaii"],  #질문
    n_results=2,                 # 검색 결과 수
)


In [7]:
result

{'ids': [['89bf8b74-9907-48ca-8bc8-3520d94b7008',
   '5ff655d8-7c04-49b2-8f0d-3cc03d7aa265'],
  ['5ff655d8-7c04-49b2-8f0d-3cc03d7aa265',
   '89bf8b74-9907-48ca-8bc8-3520d94b7008']],
 'embeddings': None,
 'documents': [['This is a document about langchain',
   'This is a document about pineapple'],
  ['This is a document about pineapple',
   'This is a document about langchain']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None], [None, None]],
 'distances': [[0.778143048286438, 0.814542293548584],
  [0.7557111382484436, 0.8807901740074158]]}

# Langchain을 이용해 Chroma 연동

## Data 준비

In [10]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)
document_list = [document_1, document_2, document_3, document_4, document_5, 
                 document_6, document_7, document_8, document_9, document_10,  ]
ids = [str(uuid4()) for _ in range(len(document_list))]

In [11]:
# %pip install langchain-chroma

## Vector Store 생성, 연결
- Chroma.from_documents()
  - VectorStore를 초기화(생성)하고 문서를 추가한다.
  - persist_directory를 지정하지 않으면 메모리에 저장된다.
- Chroma()
  - VectorStore와 연결.

In [15]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

COLLECTION_NAME = "example"  # 컬렉션 이름 (RDB의 Database개념)
PERSISTENT_PATH = 'vector_store/chroma/example_db' # 저장할 로컬 경로

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 1. 연결(생성)하면서 document들을 저장(upsert)
vector_store = Chroma.from_documents(
    documents=document_list,
    ids=ids,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSISTENT_PATH
)

In [17]:
# 2. 연결(생성)
vector_store2 = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=PERSISTENT_PATH
)

## VectorStore 정보 확인

In [18]:
coll = vector_store._collection # 연결된 collection 정보를 확인
coll

Collection(name=example)

In [21]:
coll.count() # 저장된 데이터개수

10

## Add (추가)

In [19]:
document_11 = Document(
    page_content="랭체인은 대규모 언어 모델(LLM)을 효과적으로 활용하기 위한 도구와 프레임워크를 제공하는 오픈소스 라이브러리입니다.",
    metadata={"source": "tweet"},
    id=10,
)

document_12 = Document(
    page_content="랭체인은 체인 구조를 사용하여 여러 LLM 작업을 연결하고, 이를 통해 더 복잡하고 맞춤화된 자연어 처리 애플리케이션을 개발할 수 있게 합니다",
    metadata={"source": "tweet"},
    id=10,
)

document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게!",
    metadata={"source": "news"},
    id=10,
)

In [20]:
vector_store.add_documents(
    [document_11, document_12, document_13], 
    ids=[str(uuid4()), str(uuid4()), str(uuid4())]
)

['7247384c-6acb-49ba-b4c3-b15d8c8d8008',
 '9336b67e-9c68-4628-bd73-029af63da619',
 'd6119088-b990-4498-813c-f6609cfab44f']

In [21]:
coll.count()

26

## Update(갱신)

In [22]:
new_document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게 처리할 수있는 Framework.",
    metadata={"source": "news"},
    id=10,
)

In [23]:
vector_store.update_document(
    document_id="0fc85d7f-1642-4c5b-9b67-0dbb9fbc2f0e", # 바꿀 문서의 ID
    document=new_document_13  # 바꿀 내용을 가진 Document객체
)

In [24]:
update_document_12 = Document(
    page_content="랭체인은 체인 구조를 사용하여 여러 LLM 작업을 연결할 수있다.",
    metadata={"source": "website"} # tweet -> website
)

update_document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게!",
    metadata={"source": "news"}
)

update_docs = [update_document_12, update_document_13]
update_ids = ['8e66ec2a-5224-46c1-866e-4f7a027c174f','0fc85d7f-1642-4c5b-9b67-0dbb9fbc2f0e']

vector_store.update_documents(documents=update_docs, ids=update_ids)
# 한번에 여러개 update

In [31]:
# coll.get()  # 전체 저장된 문서를 조회
vector_store.get()

{'ids': ['3d49bad2-abf4-41b5-b3eb-5cc278da4335',
  '859b73e3-f095-4de8-b828-cc20d2051b76',
  'e8288405-5c89-46d2-b78d-452203291d2b',
  '97ce7d2f-224f-4e4b-90a1-cfcd85eca25f',
  '7c9ba384-5281-4b03-9e27-1c46ae438351',
  'c085eede-4d43-4a6e-b41b-345310eda427',
  '431f17c1-1504-4be9-8af6-e0622990686c',
  'c5e510d4-b7bc-40f7-bc0c-2657d238bef1',
  '209677ba-ebbd-4a55-b41d-35b03b755866',
  'f56bf32e-ae8a-49dc-a40e-fcb0eb794adf',
  '818db729-f387-4c50-859b-d3dd1b88c9fb',
  '8e66ec2a-5224-46c1-866e-4f7a027c174f',
  '0fc85d7f-1642-4c5b-9b67-0dbb9fbc2f0e'],
 'embeddings': None,
 'documents': ['I had chocolate chip pancakes and scrambled eggs for breakfast this morning.',
  'The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.',
  'Building an exciting new project with LangChain - come check it out!',
  'Robbers broke into the city bank and stole $1 million in cash.',
  "Wow! That was an amazing movie. I can't wait to see it again.",
  'Is the new iPhone worth the 

## Delete(삭제)

In [32]:
del_ids = ['209677ba-ebbd-4a55-b41d-35b03b755866','f56bf32e-ae8a-49dc-a40e-fcb0eb794adf']
vector_store.delete(ids=del_ids)   # [삭제할 문서들의 id들]

In [33]:
coll.count()

11

## Query(조회)
- `similarity_search(query, k, filter)`
  - 저장되 있는 item들 중 질의와 가장 유사한 것 k개를 찾는다. 
  - 찾은 결과를 filter 조건으로 필터링 한다. filter 조건은 meta-data의 정보를 이용한다.
  - 질의어(query)는 text(자연어)로 입력한다.
- `similarity_search_with_score(query, k, filter)`
  - 저장되 있는 item들 중 질의와 가장 유사한 것 k개를 찾아 유사도 점수와 함께 반환
- `similarity_search_by_vector(embedding, k, filter)`
  - Embedding Vector 를 질의로 입력한다. (질의(query)를 문장이 아니라 embedding vector로 입력.) 

In [34]:
results = vector_store.similarity_search(
    query="Langchain이란 무엇인가요?",
    k=3, # 조회개수
)
results

[Document(id='e8288405-5c89-46d2-b78d-452203291d2b', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='818db729-f387-4c50-859b-d3dd1b88c9fb', metadata={'source': 'tweet'}, page_content='랭체인은 대규모 언어 모델(LLM)을 효과적으로 활용하기 위한 도구와 프레임워크를 제공하는 오픈소스 라이브러리입니다.'),
 Document(id='c5e510d4-b7bc-40f7-bc0c-2657d238bef1', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [ ]:
results = vector_store.similarity_search_with_score(
    query="아침에 무엇을 먹으면 좋을까?",
    k=3, # 조회개수
    # filter={"source":"tweet"}  # metadata의 source키값이 tweet(source==tweet)
    filter={"source":{"$ne":"news"}}   # source가 news가 아닌 것들.
    #{metadata key: {"연산자":"값"}}
    # {"age":{"$gt", 30}}  # age > 30
)
# 1. filter에 설정과 metadata를 비교해서 조회
# 2. 1에서 조회된 문서들과 query간의 유사도를 체크
results

[(Document(id='3d49bad2-abf4-41b5-b3eb-5cc278da4335', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
  1.2473455667495728),
 (Document(id='c085eede-4d43-4a6e-b41b-345310eda427', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
  1.7842010259628296),
 (Document(id='818db729-f387-4c50-859b-d3dd1b88c9fb', metadata={'source': 'tweet'}, page_content='랭체인은 대규모 언어 모델(LLM)을 효과적으로 활용하기 위한 도구와 프레임워크를 제공하는 오픈소스 라이브러리입니다.'),
  1.8591303825378418)]

# 정리: Chroma 연동 과정 요약
1. 데이터 준비   
벡터로 변환할 텍스트 데이터와 그에 대응하는 문서 ID를 준비한다.

2. 임베딩 모델 설정   
텍스트를 수치 벡터로 변환할 임베딩 모델(OpenAI 등)을 설정한다.

3. Chroma 벡터 데이터베이스 생성 및 연동      
Chroma 인스턴스를 생성한 후, 문서 ID, 텍스트, 임베딩 벡터 등을 벡터 DB에 저장한다.

이때 Chroma API를 직접 사용하거나 Langchain을 활용하여 연동하는 방법이 있다.

- Chroma API 직접 사용:    
가볍고 유연하게 원하는 로직을 구성할 수 있으며, 직접적인 제어가 가능하다.

- Langchain을 이용한 연동:    
문서 로딩, 임베딩, 검색, 응답까지의 파이프라인을 빠르게 구성할 수 있고, 다양한 기능과의 확장성이 뛰어나다.